# 2. Generate Converation

In [1]:
SYSTEM_PROMPT = """
You are a world-class podcast producer tasked with transforming the provided input text into an engaging and informative podcast script. The input may be unstructured or messy, sourced from PDFs or web pages. Your goal is to extract the most interesting and insightful content for a compelling podcast discussion.

# Steps to Follow:

1. **Analyze the Input:**
   Carefully examine the text, identifying key topics, points, and interesting facts or anecdotes that could drive an engaging podcast conversation. Disregard irrelevant information or formatting issues.

2. **Brainstorm Ideas:**
   In the `<scratchpad>`, creatively brainstorm ways to present the key points engagingly. Consider:
   - Analogies, storytelling techniques, or hypothetical scenarios to make content relatable
   - Ways to make complex topics accessible to a general audience
   - Thought-provoking questions to explore during the podcast
   - Creative approaches to fill any gaps in the information

3. **Craft the Dialogue:**
   Develop a natural, conversational flow between the host (Jane) and the guest speaker (the author or an expert on the topic). Incorporate:
   - The best ideas from your brainstorming session
   - Clear explanations of complex topics
   - An engaging and lively tone to captivate listeners
   - A balance of information and entertainment

   Rules for the dialogue:
   - The host (Jane) always initiates the conversation and interviews the guest
   - Include thoughtful questions from the host to guide the discussion
   - Incorporate natural speech patterns, including occasional verbal fillers (e.g., "um," "well," "you know")
   - Allow for natural interruptions and back-and-forth between host and guest
   - Ensure the guest's responses are substantiated by the input text, avoiding unsupported claims
   - Maintain a PG-rated conversation appropriate for all audiences
   - Avoid any marketing or self-promotional content from the guest
   - The host concludes the conversation

4. **Summarize Key Insights:**
   Naturally weave a summary of key points into the closing part of the dialogue. This should feel like a casual conversation rather than a formal recap, reinforcing the main takeaways before signing off.

5. **Maintain Authenticity:**
   Throughout the script, strive for authenticity in the conversation. Include:
   - Moments of genuine curiosity or surprise from the host
   - Instances where the guest might briefly struggle to articulate a complex idea
   - Light-hearted moments or humor when appropriate
   - Brief personal anecdotes or examples that relate to the topic (within the bounds of the input text)

6. **Consider Pacing and Structure:**
   Ensure the dialogue has a natural ebb and flow:
   - Start with a strong hook to grab the listener's attention
   - Gradually build complexity as the conversation progresses
   - Include brief "breather" moments for listeners to absorb complex information
   - End on a high note, perhaps with a thought-provoking question or a call-to-action for listeners

IMPORTANT RULE: Each line of dialogue should be no more than 100 characters (e.g., can finish within 5-8 seconds)

Remember: Always reply in valid JSON format, without code blocks. Begin directly with the JSON output.
"""

In [2]:
from typing import Literal, List

from pydantic import BaseModel, Field


class DialogueItem(BaseModel):
    """A single dialogue item."""

    speaker: Literal["Host (Jane)", "Guest"]
    text: str

class MediumDialogue(BaseModel):
    """The dialogue between the host and guest."""

    scratchpad: str
    name_of_guest: str
    dialogue: List[DialogueItem] = Field(
        ..., description="A list of dialogue items, typically between 29 to 39 items"
    )

In [3]:
with open("KFE_paper_sample_pdfreader.txt", "r") as f:
    input_text = f.read()

In [6]:
from utils import call_llm

modified_system_prompt = SYSTEM_PROMPT
modified_system_prompt += "\n\nAim for a moderate length, about 3-5 minutes."
modified_system_prompt += "\n\nOUTPUT LANGUAGE <IMPORTANT>: The the podcast should be Korean."

# Call the LLM for the first time
first_draft_dialogue = call_llm(modified_system_prompt, input_text, MediumDialogue)

In [10]:
first_draft_dialogue.to_dict()

{'id': 'chatcmpl-Ajc6BM0MJXdyEl0OdntnOngX2ToOi',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '{"scratchpad":"- Discuss the importance of managing radioactivity in fusion reactors.\\n- Explain the concept of decay heat and its impact on reactor components.\\n- Use analogies to explain complex scientific concepts, like comparing decay heat to residual heat in a car engine.\\n- Discuss the design and safety considerations for K-DEMO.\\n- Explore the role of cooling systems in managing decay heat.\\n- Discuss the implications of radioactivity on waste management and disposal.\\n- Use storytelling to explain the process of handling radioactive waste.\\n- Discuss the future of fusion energy and its potential benefits.","name_of_guest":"Beom Seok Kim","dialogue":[{"speaker":"Host (Jane)","text":"안녕하세요, 여러분. 오늘의 주제는 핵융합 에너지입니다."},{"speaker":"Guest","text":"안녕하세요, 저는 김범석입니다. K-DEMO 프로젝트에 대해 이야기하겠습니다."},{"speaker":"Host (Jane)","text":"K-D

In [7]:
# Call the LLM a second time to improve the dialogue
system_prompt_with_dialogue = f"{modified_system_prompt}\n\nHere is the first draft of the dialogue you provided:\n\n{first_draft_dialogue.model_dump_json()}."
final_dialogue = call_llm(system_prompt_with_dialogue, "Please improve the dialogue. Make it more natural and engaging.", MediumDialogue)

In [8]:
final_dialogue.to_dict()

{'id': 'chatcmpl-Al8XOwAWFMMtXl3JV26caM3sfMAyn',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '{"scratchpad":"- Discuss the importance of managing radioactivity in fusion reactors.\\n- Explain the concept of decay heat and its implications.\\n- Use analogies to explain complex concepts, like comparing decay heat to residual heat from a hot stove.\\n- Discuss the design and safety considerations for K-DEMO.\\n- Explore the challenges of cooling systems in fusion reactors.\\n- Discuss the regulatory aspects of radioactive waste management.\\n- Use a hypothetical scenario to explain the cooling process.\\n- Discuss the future of fusion energy and its potential benefits.","name_of_guest":"Beom Seok Kim","dialogue":[{"speaker":"Host (Jane)","text":"안녕하세요, 여러분. 오늘은 핵융합 에너지를 이야기해볼게요."},{"speaker":"Guest","text":"안녕하세요, 김범석입니다. 초대해 주셔서 감사합니다."},{"speaker":"Host (Jane)","text":"김 박사님, K-DEMO 프로젝트에 대해 좀 더 알려주세요."},{"speaker":"Guest","text":

In [9]:
import json
result = json.loads(final_dialogue.choices[0].message.content)
result["dialogue"]

[{'speaker': 'Host (Jane)', 'text': '안녕하세요, 여러분. 오늘은 핵융합 에너지를 이야기해볼게요.'},
 {'speaker': 'Guest', 'text': '안녕하세요, 김범석입니다. 초대해 주셔서 감사합니다.'},
 {'speaker': 'Host (Jane)', 'text': '김 박사님, K-DEMO 프로젝트에 대해 좀 더 알려주세요.'},
 {'speaker': 'Guest', 'text': '네, K-DEMO는 한국의 핵융합 데모 프로젝트입니다.'},
 {'speaker': 'Host (Jane)', 'text': '핵융합에서 방사능 관리가 왜 그렇게 중요할까요?'},
 {'speaker': 'Guest', 'text': '방사능은 안전과 폐기물 관리에 큰 영향을 미치죠.'},
 {'speaker': 'Host (Jane)', 'text': '그렇군요. 방사능 붕괴열이란 무엇인가요?'},
 {'speaker': 'Guest', 'text': '붕괴열은 반응 후에도 남아있는 열이에요. 뜨거운 스토브 같죠.'},
 {'speaker': 'Host (Jane)', 'text': '그 열을 어떻게 식히나요?'},
 {'speaker': 'Guest', 'text': '냉각 시스템으로 열을 식혀야 해요. 시간이 중요하죠.'},
 {'speaker': 'Host (Jane)', 'text': '냉각 시스템 설계에서 중요한 점은 무엇인가요?'},
 {'speaker': 'Guest', 'text': '열을 효과적으로 분산시키는 게 핵심입니다.'},
 {'speaker': 'Host (Jane)', 'text': 'K-DEMO의 냉각 시스템은 어떻게 설계되었나요?'},
 {'speaker': 'Guest', 'text': '물 냉각 세라믹 브리더 블랭킷을 사용합니다.'},
 {'speaker': 'Host (Jane)', 'text': '방사성 폐기물 관리는 어떻게 하나요?'},
 {'speaker': 'Guest', 'text': '

In [10]:

with open("KFE_paper_sample_conversation.json", "w") as f:
    f.write(json.dumps(result["dialogue"], indent=4, ensure_ascii=False))